# Analysis of libjpeg - downsampling

**Author:** Martin Beneš

This notebook contains forensic analysis of chroma downsampling in various libjpeg versions. Both RGB and grayscale are tested separately. All the other parameters are kept default.

In [1]:
# versions to test
versions = ['6b','turbo210','7','8','8a','8b','8c','8d','9','9a','9b','9c','9d','9e']

# default versions
import jpeglib
v_arbitrary = '9e' # arbitrary version for decompression
jpeglib.version.set(v_arbitrary)
d_flags = ['+DO_FANCY_UPSAMPLING']

# random subsample size
N_samples = 10

# database path
from pathlib import Path
db_path = Path.home() / 'Datasets'

# sampling factor
samp_factors = [
    ((1,1),(1,1),(1,1)), # 4:4:4
    ((1,2),(1,2),(1,2)),
    ((2,1),(2,1),(2,1)),
    
    ((1,2),(1,1),(1,1)), # 4:4:0
    ((2,2),(2,1),(2,1)),
    ((1,4),(1,2),(1,2)),
    ((1,2),(1,2),(1,1)),   # Cb 4:4:4 Cr 4:4:0
    ((1,2),(1,1),(1,2)),   # Cb 4:4:0 Cr 4:4:4
    
    ((2,1),(1,1),(1,1)), # 4:2:2
    ((2,2),(1,2),(1,2)),
    ((2,1),(2,1),(1,1)),   # Cb 4:4:4 Cr 4:2:2
    ((2,1),(1,1),(2,1)),   # Cb 4:2:2 Cr 4:4:4
    
    ((2,2),(1,1),(1,1)), # 4:2:0
    ((2,2),(2,1),(1,1)),   # Cb 4:4:0 Cr 4:2:0
    ((2,2),(1,1),(2,1)),   # Cb 4:2:0 Cr 4:4:0
    ((2,2),(1,2),(1,1)),   # Cb 4:2:2 Cr 4:2:0
    ((2,2),(1,1),(1,2)),   # Cb 4:2:0 Cr 4:2:2
    ((2,2),(2,2),(1,1)),   # Cb 4:4:4 Cr 4:2:0
    ((2,2),(2,2),(2,1)),   # Cb 4:4:4 Cr 4:4:0
    ((2,2),(2,2),(1,2)),   # Cb 4:4:4 Cr 4:2:2
    ((2,2),(1,1),(2,2)),   # Cb 4:2:0 Cr 4:4:4
    ((2,2),(2,1),(2,2)),   # Cb 4:4:0 Cr 4:4:4
    ((2,2),(1,2),(2,2)),   # Cb 4:2:2 Cr 4:4:4
    
    ((4,1),(1,1),(1,1)), # 4:1:1
    ((4,1),(2,1),(1,1)),   # Cb 4:2:2 Cr 4:1:1
    ((4,1),(1,1),(2,1)),   # Cb 4:1:1 Cr 4:2:2
    
    ((4,2),(1,1),(1,1)), # 4:1:0
    
    ((1,4),(1,1),(1,1)), # 1:0.5:0
    ((1,4),(1,2),(1,1)),
    
    ((2,4),(1,1),(1,1)), # 2:0.5:0
    
    ((3,1),(1,1),(1,1)), # 3:1:1
    ((3,1),(3,1),(1,1)),   # Cb 4:4:4 Cr 3:1:1
    ((3,1),(1,1),(3,1)),   # Cb 3:1:1 Cr 4:4:4
    ((3,2),(3,1),(1,1)), # 3:3:0
    ((3,2),(1,2),(1,2)), # 3:1:1
]

# checkerboard
import numpy as np
def checkerboard(boardsize, tilesize, channels=3):
    board = np.zeros([*boardsize, channels], dtype=np.uint8)
    for i in range(boardsize[0]):
        for j in range(boardsize[1]):
            if (i//tilesize[0]) % 2 == (j//tilesize[1]) % 2:
                board[i,j] = 255
    return board

## Load ALASKA

Load ALASKA2 database consisting of 70000 colored images. In this case we have uncompressed version of shape 256x256. You can find the scripts to download it [here](https://alaska.utt.fr/).

In [2]:
# Load ALASKA2 database
import os
alaska_path = db_path / 'ALASKA_v2_TIFF_256_COLOR'
alaska_names = [alaska_path / f for f in os.listdir(alaska_path)]
print("Loaded ALASKA2 database with", len(alaska_names), "images.")

# sample without replacement
import random
random.seed(42) # answer to everything
alaska_names_sub = random.sample(alaska_names, N_samples-2)

# choose most and least saturated
import matplotlib.pyplot as plt
#most,least = (None,0),(None,0)
#for i,f in enumerate(alaska_names):
#    if i % 500 == 0: print(i, '/', len(alaska_names), '         ', end='\r')
#    if str(f).split('.')[-1] != 'tif': continue
#    x = plt.imread(str(f))
#    xmin,xmax = (x == 0).sum(),(x == 255).sum()
#    if xmin > least[1]: least = (f,xmin)
#    if xmax > most[1]: most = (f,xmax)
most,least = (alaska_path / '10343.tif',98491),(alaska_path / '05887.tif', 78128)
# add them
alaska_names_sub.append(most[0])
alaska_names_sub.append(least[0])

# load the image with PIL
import numpy as np
alaska = np.array([plt.imread(f) for f in alaska_names_sub])

# append checkerboard
for tilesize in [(4,4),(7,7),(8,8),(15,15),(16,16)]:
    alaska = np.append(alaska, np.expand_dims(checkerboard((256, 256), tilesize, 3), 0), 0)

print("Input shape", alaska.shape)

Loaded ALASKA2 database with 80004 images.
Input shape (15, 256, 256, 3)


## Compression with fancy upsampling as compression and decompression flag

### Compression

In [3]:
# images recompressed by each version
import tempfile
images_rgb = {'version': [], 'samp_factor': [], 'Y': [], 'Cb': [], 'Cr': [], 'image': []}
kw = {'qt': 75, 'flags': ['DO_FANCY_UPSAMPLING'], 'in_color_space': 'JCS_RGB'}

with tempfile.TemporaryDirectory() as tmp:
    
    # iterate versions
    for i,v_compress in enumerate(versions):
        
        # iterate samp factors
        for samp_factor in samp_factors:
            
            # compress each image with version
            fnames = [str(Path(tmp) / f'{i}.jpeg') for i in range(alaska.shape[0])]
            with jpeglib.version(v_compress):
                for i,fname in enumerate(fnames):
                    im = jpeglib.from_spatial(alaska[i])
                    im.samp_factor = samp_factor
                    im.write_spatial(fname, flags=['+DO_FANCY_UPSAMPLING'])
        
            # decompress with single (arbitrary) version
            with jpeglib.version(v_arbitrary):
                images_rgb['version'].append(v_compress)
                images_rgb['samp_factor'].append(samp_factor)
                images_rgb['image'].append(np.array([
                    jpeglib.read_spatial(fname, flags=d_flags).spatial for fname in fnames
                ]))
                images_rgb['Y'].append([
                    jpeglib.read_dct(fname).Y for fname in fnames
                ])
                images_rgb['Cb'].append([
                    jpeglib.read_dct(fname).Cb for fname in fnames
                ])
                images_rgb['Cr'].append([
                    jpeglib.read_dct(fname).Cr for fname in fnames
                ])

# dataframe
import pandas as pd
images_rgb = pd.DataFrame(images_rgb)

### N-to-N comparison

In [4]:
# L1 distance metric
from scipy.spatial.distance import pdist, squareform
mismatch = lambda x1,x2: (np.abs(x1.astype(np.int32) - x2.astype(np.int32)) != 0).mean()

# get distance matrix
def get_distmat(x):
    # images to distance matrix
    images_x_list = np.array([list(i) for i in x.to_list()], dtype=object)
    images_x_list = images_x_list[:80,:80]
    images_x_list = images_x_list.reshape(len(versions), -1)
    dists_x = pdist(images_x_list, mismatch)
    return squareform(dists_x)

# images by sampling factor
distmats_rgb = {}
distmats_Y = {}
distmats_Cb = {}
distmats_Cr = {}
for samp_factor in samp_factors:
    images_rgb_sf = images_rgb[images_rgb.samp_factor == samp_factor]
    
    # images to distance metric
    distmats_rgb[samp_factor] = get_distmat(images_rgb_sf.image)
    distmats_Y[samp_factor] = get_distmat(images_rgb_sf.Y)
    distmats_Cb[samp_factor] = get_distmat(images_rgb_sf.Cb)
    distmats_Cr[samp_factor] = get_distmat(images_rgb_sf.Cr)
    
    print(
        samp_factor,
        ((distmats_rgb[samp_factor] == 0) == (distmats_Y[samp_factor] == 0)).all(),
        ((distmats_rgb[samp_factor] == 0) == (distmats_Cb[samp_factor] == 0)).all(),
        ((distmats_rgb[samp_factor] == 0) == (distmats_Cr[samp_factor] == 0)).all(),
    )

((1, 1), (1, 1), (1, 1)) False True True
((1, 2), (1, 2), (1, 2)) False True True
((2, 1), (2, 1), (2, 1)) False True True
((1, 2), (1, 1), (1, 1)) False True True
((2, 2), (2, 1), (2, 1)) False True True
((1, 4), (1, 2), (1, 2)) False True True
((1, 2), (1, 2), (1, 1)) False False True
((1, 2), (1, 1), (1, 2)) False True False
((2, 1), (1, 1), (1, 1)) False True True
((2, 2), (1, 2), (1, 2)) False True True
((2, 1), (2, 1), (1, 1)) False False True
((2, 1), (1, 1), (2, 1)) False True False
((2, 2), (1, 1), (1, 1)) False True True
((2, 2), (2, 1), (1, 1)) False True True
((2, 2), (1, 1), (2, 1)) False True True
((2, 2), (1, 2), (1, 1)) False True True
((2, 2), (1, 1), (1, 2)) False True True
((2, 2), (2, 2), (1, 1)) False False True
((2, 2), (2, 2), (2, 1)) False False True
((2, 2), (2, 2), (1, 2)) False False True
((2, 2), (1, 1), (2, 2)) False True False
((2, 2), (2, 1), (2, 2)) False True False
((2, 2), (1, 2), (2, 2)) False True False
((4, 1), (1, 1), (1, 1)) False True True
((4, 1

There is no difference between DCT and spatial domain. Thus behavior in decompression is the same in all the versions.

### Clustering

In [5]:
from sklearn.cluster import AgglomerativeClustering
# cluster by sampling factor
for samp_factor in samp_factors:
    for k in range(1,6):
        agnes = AgglomerativeClustering(n_clusters=k, linkage='average', affinity='precomputed')
        agnes.fit(distmats_rgb[samp_factor])
    
        # compute heterogenity metric (sum of distances)
        heterogenity = np.sum([ distmats_rgb[samp_factor][i,j]
                 for group in np.unique(agnes.labels_)
                 for i in np.where(agnes.labels_ == group)[0]
                 for j in np.where(agnes.labels_ == group)[0] ])
    
        # homogenous clusters
        if heterogenity == 0: break
    print(samp_factor, ":")
    print(" ->", k, "classes:", *[[versions[i] for i in np.where(agnes.labels_ == cl)[0]] for cl in np.unique(agnes.labels_)])

((1, 1), (1, 1), (1, 1)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((1, 2), (1, 2), (1, 2)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((2, 1), (2, 1), (2, 1)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((1, 2), (1, 1), (1, 1)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((2, 2), (2, 1), (2, 1)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((1, 4), (1, 2), (1, 2)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((1, 2), (1, 2), (1, 1)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((1, 2), (1, 1), (1, 2)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9

In [6]:
from sklearn.cluster import AgglomerativeClustering
# cluster by sampling factor
for samp_factor in samp_factors:
    for k in range(1,6):
        agnes = AgglomerativeClustering(n_clusters=k, linkage='average', affinity='precomputed')
        agnes.fit(distmats_Cb[samp_factor])
    
        # compute heterogenity metric (sum of distances)
        heterogenity = np.sum([ distmats_Cb[samp_factor][i,j]
                 for group in np.unique(agnes.labels_)
                 for i in np.where(agnes.labels_ == group)[0]
                 for j in np.where(agnes.labels_ == group)[0] ])
    
        # homogenous clusters
        if heterogenity == 0: break
    print(samp_factor, ":")
    print(" ->", k, "classes:", *[[versions[i] for i in np.where(agnes.labels_ == cl)[0]] for cl in np.unique(agnes.labels_)])

((1, 1), (1, 1), (1, 1)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((1, 2), (1, 2), (1, 2)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((2, 1), (2, 1), (2, 1)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((1, 2), (1, 1), (1, 1)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((2, 2), (2, 1), (2, 1)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((1, 4), (1, 2), (1, 2)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((1, 2), (1, 2), (1, 1)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((1, 2), (1, 1), (1, 2)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9'

In [7]:
from sklearn.cluster import AgglomerativeClustering
# cluster by sampling factor
for samp_factor in samp_factors:
    for k in range(1,6):
        agnes = AgglomerativeClustering(n_clusters=k, linkage='average', affinity='precomputed')
        agnes.fit(distmats_Cr[samp_factor])
    
        # compute heterogenity metric (sum of distances)
        heterogenity = np.sum([ distmats_Cr[samp_factor][i,j]
                 for group in np.unique(agnes.labels_)
                 for i in np.where(agnes.labels_ == group)[0]
                 for j in np.where(agnes.labels_ == group)[0] ])
    
        # homogenous clusters
        if heterogenity == 0: break
    print(samp_factor, ":")
    print(" ->", k, "classes:", *[[versions[i] for i in np.where(agnes.labels_ == cl)[0]] for cl in np.unique(agnes.labels_)])

((1, 1), (1, 1), (1, 1)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((1, 2), (1, 2), (1, 2)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((2, 1), (2, 1), (2, 1)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e']
((1, 2), (1, 1), (1, 1)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((2, 2), (2, 1), (2, 1)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((1, 4), (1, 2), (1, 2)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((1, 2), (1, 2), (1, 1)) :
 -> 3 classes: ['7', '8', '8a', '8b', '8c', '8d', '9', '9a', '9b', '9c', '9d'] ['9e'] ['6b', 'turbo210']
((1, 2), (1, 1), (1, 2)) :
 -> 2 classes: ['6b', 'turbo210', '7', '8', '8a', '8

Mismatch in 7 for factors 2 and 4. 9e mismatches from 9d in all cases.